# 🌬️ HLCA Lung (core) snRNA‑seq Processing Workflow

## 📖 Introduction & Data
**Dataset**: HLCA core (healthy lung samples) • 584 944 cells / 27 957 genes  
**Paper**: Sikkema *et al.* (2023) *Nat Med*  
**Collection**: <https://cellxgene.cziscience.com/collections/6f6d381a-7701-4781-935c-db10d30de293>

### Outputs
| File | Description |
| --- | --- |
| `HLCA_core_raw.h5ad` | Raw counts AnnData |
| `HLCA_core_healthy_RA_expr_gene_withPos.h5ad`, `...LP...` | Expr + coords subsets |
| `HLCA_core_healthy_RA_pc.h5ad`, `...LP_pc.h5ad` | Protein‑coding subsets |
| `.cov` | scDRS covariates |
| Gene‑type plots | PNG bar plots |
| Gene lists | CSV (all / pc) |
| `HLCA_core_celltypes_levels.txt` | Cell‑type hierarchy |

*All paths below are relative (`DATA_DIR`, `OUTPUT_DIR`).*


## 🔧 Environment Setup

In [ ]:
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

## 📂 Define Paths

In [ ]:
DATA_DIR   = Path('data/HLCA')        # contains HLCA_core.h5ad
OUTPUT_DIR = Path('output/HLCA')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

HLCA_CORE = DATA_DIR/'HLCA_core.h5ad'
GENE_MATRIX = Path('data')/'geneMatrix.tsv.gz'  # 56k gene coords

## 📑 Load HLCA Core & Replace with Raw Counts

In [ ]:
adata = sc.read_h5ad(HLCA_CORE)
adata.X = adata.raw.X
adata.raw = None
adata.write(OUTPUT_DIR/'HLCA_core_raw.h5ad')
print(adata)

## 🧬 Keep Genes with Genomic Coordinates

In [ ]:
gene_coords = pd.read_csv(GENE_MATRIX, sep='\t', compression='infer')
common = gene_coords['Gene'].astype(str).intersection(adata.var_names)
adata = adata[:, adata.var_names.isin(common)].copy()
print('Genes retained:', adata.n_vars)
for col in ['feature_biotype','feature_is_filtered','feature_reference','feature_length']:
    if col in adata.var.columns:
        adata.var.drop(columns=col, inplace=True)

## 🗂️ Export Cell‑Type Level Hierarchy

In [ ]:
cols_lvl = ['ann_finest_level','ann_level_1','ann_level_2','ann_level_3','ann_level_4','ann_level_5','cell_type','tissue']
adata.obs[cols_lvl].drop_duplicates().to_csv(OUTPUT_DIR/'HLCA_core_celltypes_levels.txt', sep='\t', index=False)

## 🩺 Subset to Healthy Samples

In [ ]:
adata_h = adata[adata.obs['lung_condition']=='Healthy'].copy()

## 🌳 Process Respiratory Airway (RA) & Lung Parenchyma (LP)

In [ ]:
def process_tissue(a, tissue_tag, out_prefix):
    sub = a[a.obs['tissue']==tissue_tag].copy()
    # Minimal obs cols
    sub.obs = sub.obs[['cell_type','sample','sex','age_or_mean_of_age_range']]
    sub.obs.rename(columns={'age_or_mean_of_age_range':'age'}, inplace=True)
    # Sex to numeric (female=0, male=1)
    sub.obs['sex'] = sub.obs['sex'].map({'female':0,'male':1})
    # Filter cell types <20
    counts = sub.obs['cell_type'].value_counts()
    keep = counts[counts>=20].index
    sub = sub[sub.obs['cell_type'].isin(keep)].copy()
    # Save expression+pos h5ad
    sub.write(OUTPUT_DIR/f'{out_prefix}_expr_gene_withPos.h5ad')
    # Covariate file
    cov = pd.DataFrame(index=sub.obs.index)
    cov['const']=1
    cov['n_genes']=(sub.X>0).sum(axis=1)
    cov['sex']=sub.obs['sex']; cov['age']=sub.obs['age']
    for samp in sorted(sub.obs['sample'].unique()):
        if samp!='356C_0h':
            cov[f'donor_{samp}']=(sub.obs['sample']==samp).astype(int)
    cov.to_csv(OUTPUT_DIR/f'{out_prefix}_expr_gene_withPos.cov',sep='\t')
    # Protein‑coding subset
    pc_genes = gene_coords[gene_coords['gene_type']=='protein_coding']['Gene']
    sub_pc = sub[:, sub.var_names.isin(pc_genes)].copy()
    sub_pc.write(OUTPUT_DIR/f'{out_prefix}_pc.h5ad')
    # Gene‑type plot
    gt_counts = gene_coords[gene_coords['Gene'].isin(sub.var_names)]['gene_type'].value_counts()
    ax = gt_counts.plot(kind='bar', figsize=(8,6))
    ax.set_ylabel('Genes'); ax.set_xlabel('Gene type');
    plt.xticks(rotation=45, ha='right'); plt.tight_layout()
    plt.savefig(OUTPUT_DIR/f'{out_prefix}_gene_type_distribution.png')
    plt.close()
    return sub, sub_pc

adata_RA, adata_RA_pc = process_tissue(adata_h,'respiratory airway','HLCA_core_healthy_RA')
adata_LP, adata_LP_pc = process_tissue(adata_h,'lung parenchyma','HLCA_core_healthy_LP')

## 🗃️ Export Combined Gene Lists

In [ ]:
pd.Series(adata_h.var_names,name='ensgid').to_csv(OUTPUT_DIR/'HLCA_core_healthy_allgene_list.csv',index=False)
# PC list already saved in process_tissue (LP example)
